In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_excel('Sample - Superstore.xls')
print(df.head())
print(df.shape)

   Row ID        Order ID Order Date  Ship Date       Ship Mode Customer ID  \
0       1  CA-2016-152156 2016-11-08 2016-11-11    Second Class    CG-12520   
1       2  CA-2016-152156 2016-11-08 2016-11-11    Second Class    CG-12520   
2       3  CA-2016-138688 2016-06-12 2016-06-16    Second Class    DV-13045   
3       4  US-2015-108966 2015-10-11 2015-10-18  Standard Class    SO-20335   
4       5  US-2015-108966 2015-10-11 2015-10-18  Standard Class    SO-20335   

     Customer Name    Segment        Country             City  ...  \
0      Claire Gute   Consumer  United States        Henderson  ...   
1      Claire Gute   Consumer  United States        Henderson  ...   
2  Darrin Van Huff  Corporate  United States      Los Angeles  ...   
3   Sean O'Donnell   Consumer  United States  Fort Lauderdale  ...   
4   Sean O'Donnell   Consumer  United States  Fort Lauderdale  ...   

  Postal Code  Region       Product ID         Category Sub-Category  \
0       42420   South  FUR-BO-10

In [3]:
df['year'] = df['Order Date'].dt.year
print(df['year'].value_counts())

df['quarter'] = df['Order Date'].dt.quarter
print(df['quarter'].value_counts())

df['month'] = df['Order Date'].dt.month
print(df['month'].value_counts())





year
2017    3312
2016    2587
2015    2102
2014    1993
Name: count, dtype: int64
quarter
4    3698
3    2799
2    2120
1    1377
Name: count, dtype: int64
month
11    1471
12    1408
9     1383
10     819
5      735
6      717
7      710
8      706
3      696
4      668
1      381
2      300
Name: count, dtype: int64


In [4]:
df['weekday'] = df['Order Date'].dt.weekday
print(df['weekday'].value_counts())

weekday
0    1871
4    1818
6    1710
5    1655
3    1463
1    1106
2     371
Name: count, dtype: int64


In [17]:
# Question 127: Find top 5 customers by profit in each segment.
profit_by_segment = df.groupby(['Segment', 'Customer ID'])['Profit'].sum()
top_5_customers = profit_by_segment.groupby('Segment').nlargest(5)
print(top_5_customers)

Segment      Segment      Customer ID
Consumer     Consumer     RB-19360       6976.0959
                          SC-20095       5757.4119
                          HL-15040       5622.4292
                          AB-10105       5444.8055
                          CM-12385       3899.8904
Corporate    Corporate    TC-20980       8981.3239
                          KD-16495       3038.6254
                          BS-11365       2616.0644
                          HM-14860       2437.9836
                          TS-21370       2371.7144
Home Office  Home Office  TA-21385       4703.7883
                          DR-12940       2869.0760
                          NM-18445       2751.6849
                          AB-10060       2054.5885
                          SR-20740       1990.4244
Name: Profit, dtype: float64


In [19]:
# Question 128: Find top 5 loss-making customers.
loss_by_customer = df.groupby('Customer ID')['Profit'].sum()
top_5_loss_making = loss_by_customer.nsmallest(5)
print(top_5_loss_making)

Customer ID
CS-12505   -6626.3895
GT-14635   -4108.6589
LF-17185   -3583.9770
SR-20425   -3333.9144
HG-14965   -2797.9635
Name: Profit, dtype: float64


In [22]:
# Question 129: Find customers with largest order value.
max_order_value = df.groupby('Customer ID')['Sales'].max().nlargest(5)
print(max_order_value)

Customer ID
SM-20320    22638.480
TC-20980    17499.950
RB-19360    13999.960
TA-21385    11199.968
HL-15040    10499.970
Name: Sales, dtype: float64


In [23]:
# Question 130: Find customers with smallest order value.
min_order_value = df.groupby('Customer ID')['Sales'].min().nsmallest(5)
print(min_order_value)

Customer ID
ZC-21910    0.444
RS-19765    0.556
KB-16600    0.836
JO-15280    0.852
HG-14965    0.876
Name: Sales, dtype: float64


In [25]:
# Question 131: Find number of unique customers each year.
unique_customer_each_year = df.groupby('year')['Customer ID'].nunique()
print(unique_customer_each_year)

year
2014    595
2015    573
2016    638
2017    693
Name: Customer ID, dtype: int64


In [26]:
# Question 132: Find retention rate year over year.
retention_rate = df.groupby('year')['Customer ID'].nunique().pct_change()
print(retention_rate)

year
2014         NaN
2015   -0.036975
2016    0.113438
2017    0.086207
Name: Customer ID, dtype: float64


In [ ]:
# Question 133: Find churned customers (present last year, not this year).
# churned customers -> customers who were present last year but not this year

years_sorted = sorted(df['year'].unique())
print(f"Years: {years_sorted}")

this_year = years_sorted[-1]
last_year = years_sorted[-2]

# df.loc[df['year'] == this_year, 'Customer ID']
customers_this_year = set(df[df['year'] == this_year]['Customer ID'].unique())
customers_last_year = set(df[df['year'] == last_year]['Customer ID'].unique())
print(f"Customers this year: {len(customers_this_year)}")
print(f"Customers last year: {len(customers_last_year)}")
churned_customers = customers_last_year - customers_this_year  
print(f"Churned customers: {len(churned_customers)}")

# df[df['Customer ID'].isin(churned_customers) & (df['year'] == last_year)]
churned_df = pd.DataFrame(churned_customers, columns=['Customer ID'])
print(churned_df)

Years: [np.int32(2014), np.int32(2015), np.int32(2016), np.int32(2017)]
Customers this year: 693
Customers last year: 638
Churned customers: 80
   Customer ID
0     TC-20980
1     JL-15850
2     VD-21670
3     JS-15880
4     MC-17425
..         ...
75    CM-12235
76    JJ-15445
77    CS-12490
78    BS-11665
79    AF-10885

[80 rows x 1 columns]


In [10]:
# Question 134: Find new customers (present this year, not last year).

new_customers = customers_this_year - customers_last_year
print(f"New total customers: {len(new_customers)}")
new_customers_df = pd.DataFrame(new_customers, columns=['Customer ID'])
print(new_customers_df)

New total customers: 135
    Customer ID
0      TM-21490
1      FG-14260
2      LT-16765
3      PO-19195
4      MS-17980
..          ...
130    MW-18235
131    MR-17545
132    YS-21880
133    EM-13960
134    TC-21145

[135 rows x 1 columns]


In [ ]:
# Question 135: Find sales contribution of top 10 customers.
# IMP

# working with series
customers_sales = df.groupby('Customer ID')['Sales'].sum()
top_10_customers = customers_sales.nlargest(10)
contribution = top_10_customers.sum() / customers_sales.sum() * 100
print(f"contribution of top 10 customers: {contribution:.2f}%")


# working with dataframe, more flexible
# 1️⃣ Find total sales by each customer
# customer_sales = df.groupby('Customer ID')['Sales'].sum().reset_index()

# # 2️⃣ Sort customers by total sales (descending order)
# customer_sales = customer_sales.sort_values(by='Sales', ascending=False)

# # 3️⃣ Take top 10 customers
# top10 = customer_sales.head(10)

# # 4️⃣ Calculate total company sales
# total_sales = df['Sales'].sum()

# # 5️⃣ Calculate contribution of these top 10 customers
# top10_contribution = top10['Sales'].sum() / total_sales * 100

# # 6️⃣ Display result
# print("Top 10 Customers by Total Sales:\n", top10)
# print(f"\nSales Contribution of Top 10 Customers: {top10_contribution:.2f}%")

contribution of top 10 customers: 6.70%


In [15]:
# Question 136: Find profit contribution of top 10 customers.
customers_profit = df.groupby('Customer ID')['Profit'].sum()
top_10_profit_customers = customers_profit.nlargest(10)
profit_contribution = top_10_profit_customers.sum() / customers_profit.sum() * 100
print(f"Profit contribution of top 10 customers: {profit_contribution:.2f}%")

Profit contribution of top 10 customers: 17.52%


In [20]:
# Question 137: Find customers who purchased from all categories.
total_categories = df['Category'].nunique()
print(f"Total categories: {total_categories}")
categories_per_customer = df.groupby('Customer ID')['Category'].nunique()
customer_per_all_categories = categories_per_customer[categories_per_customer == total_categories]
print(customer_per_all_categories)

Total categories: 3
Customer ID
AA-10315    3
AA-10375    3
AA-10480    3
AA-10645    3
AB-10015    3
           ..
XP-21865    3
YC-21895    3
YS-21880    3
ZC-21910    3
ZD-21925    3
Name: Category, Length: 617, dtype: int64


In [ ]:
# Question 138: Find customers who purchased only furniture.
# IMP
categories_per_customer = df.groupby('Customer ID')['Category'].unique()
print(categories_per_customer)
only_furniture_customers = categories_per_customer[categories_per_customer.apply(lambda x: set(x) == {'Furniture'})]
print(only_furniture_customers)

Customer ID
AA-10315    [Office Supplies, Technology, Furniture]
AA-10375    [Office Supplies, Furniture, Technology]
AA-10480    [Office Supplies, Furniture, Technology]
AA-10645    [Furniture, Technology, Office Supplies]
AB-10015    [Furniture, Office Supplies, Technology]
                              ...                   
XP-21865    [Technology, Office Supplies, Furniture]
YC-21895    [Technology, Office Supplies, Furniture]
YS-21880    [Office Supplies, Furniture, Technology]
ZC-21910    [Technology, Office Supplies, Furniture]
ZD-21925    [Office Supplies, Technology, Furniture]
Name: Category, Length: 793, dtype: object
Customer ID
AO-10810    [Furniture]
PH-18790    [Furniture]
SH-20635    [Furniture]
Name: Category, dtype: object


In [27]:
# Question 139: Find customers who purchased only technology.
only_technology_customers = categories_per_customer[categories_per_customer.apply(lambda x: set(x) == {'Technology'})]
print(only_technology_customers)

Customer ID
JR-15700    [Technology]
Name: Category, dtype: object


In [28]:
# Question 140: Find customers who purchased only office supplies.
only_office_supplies_customers = categories_per_customer[categories_per_customer.apply(lambda x: set(x) == {'Office Supplies'})]
print(only_office_supplies_customers)

Customer ID
AG-10765    [Office Supplies]
AJ-10960    [Office Supplies]
BF-11080    [Office Supplies]
BO-11425    [Office Supplies]
BS-11800    [Office Supplies]
CJ-11875    [Office Supplies]
GP-14740    [Office Supplies]
HD-14785    [Office Supplies]
IG-15085    [Office Supplies]
JW-16075    [Office Supplies]
LD-16855    [Office Supplies]
RE-19405    [Office Supplies]
RH-19555    [Office Supplies]
RS-19870    [Office Supplies]
SG-20890    [Office Supplies]
TS-21085    [Office Supplies]
TT-21265    [Office Supplies]
Name: Category, dtype: object


In [ ]:
# Question 141: Find average order size per segment.
# IMP
# average_order_size_per_segment = df.groupby('Segment')['Quantity'].mean()
# print(average_order_size_per_segment)

orders_total = df.groupby(['Order ID','Segment'])['Quantity'].sum().reset_index()
avg_order_size = orders_total.groupby('Segment')['Quantity'].mean()
print(avg_order_size)

Segment
Consumer       7.548724
Corporate      7.667107
Home Office    7.419142
Name: Quantity, dtype: float64


In [40]:
# Question 142: Find average discount per segment.
avg_discount_per_segment = df.groupby('Segment')['Discount'].mean()
print(avg_discount_per_segment)

Segment
Consumer       0.158141
Corporate      0.158228
Home Office    0.147128
Name: Discount, dtype: float64


In [43]:
# Question 143: Find segment with maximum number of customers.

max_customer_segment = df.groupby('Segment')['Customer ID'].nunique().idxmax()
print(f"Segment with maximum number of customers: {max_customer_segment}")

Segment with maximum number of customers: Consumer


In [46]:
# Question 144: Find segment with minimum number of customers.
min_customer_segment = df.groupby('Segment')['Customer ID'].nunique().idxmin()
print(f"Segment with minimum number of customers: {min_customer_segment}")

Segment with minimum number of customers: Home Office


In [47]:
# Question 145: Find customer with maximum quantity purchased.
max_quantity_customer = df.groupby('Customer ID')['Quantity'].sum().idxmax()
max_quantity = df.groupby('Customer ID')['Quantity'].sum().max()
print(f"Customer with maximum quantity purchased: {max_quantity_customer} with quantity {max_quantity}")

Customer with maximum quantity purchased: JD-15895 with quantity 150


In [49]:
# Question 146: Find customer with minimum quantity purchased.
min_quantity_customer = df.groupby('Customer ID')['Quantity'].sum().idxmin()
min_quantity = df.groupby('Customer ID')['Quantity'].sum().min()
print(f"Customer with minimum quantity purchased: {min_quantity_customer} with quantity {min_quantity}")


Customer with minimum quantity purchased: AO-10810 with quantity 2


In [54]:
# Question 147: Find customer who bought most different products.
most_different_products_customer = df.groupby('Customer ID')['Product ID'].nunique().idxmax()
print(f"Customer who bought most different products: {most_different_products_customer}")

Customer who bought most different products: WB-21850


In [58]:
# Question 148: Find customer who bought most categories.
categories_customer = df.groupby('Customer ID')['Category'].nunique()
most_categories_customer = categories_customer[categories_customer == categories_customer.max()]
print(f"Customer who bought most categories: {most_categories_customer}")

Customer who bought most categories: Customer ID
AA-10315    3
AA-10375    3
AA-10480    3
AA-10645    3
AB-10015    3
           ..
XP-21865    3
YC-21895    3
YS-21880    3
ZC-21910    3
ZD-21925    3
Name: Category, Length: 617, dtype: int64


In [59]:
# Question 149: Find customer who bought most sub-categories.
subcategories_customer = df.groupby('Customer ID')['Sub-Category'].nunique()
most_subcategories_customer = subcategories_customer[subcategories_customer == subcategories_customer.max()]
print(f"Customer who bought most sub-categories: {most_subcategories_customer}")

Customer who bought most sub-categories: Customer ID
AP-10915    15
NW-18400    15
Name: Sub-Category, dtype: int64


In [60]:
# Question 150: Find customer who bought across most states.
customer_across_states = df.groupby('Customer ID')['State'].nunique()
most_states_customer = customer_across_states[customer_across_states == customer_across_states.max()]
print(f"Customer who bought across most states: {most_states_customer}")

Customer who bought across most states: Customer ID
Dp-13240    11
Name: State, dtype: int64
